<a href="https://colab.research.google.com/github/johnJay1745/CeDeFi/blob/main/SSETH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 
import numpy as np
import scipy.stats as st
import json
import requests
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from datetime import datetime

In [ ]:
def get_hist_apy(pool_id):
    data = requests.get("https://yields.llama.fi/chart/" + pool_id)
    data_json = data.json() 
    l = []
    for apy in data_json["data"]:
        l.append([apy["timestamp"],apy["apy"]])
    return pd.DataFrame(l).rename(columns = {0:"timestamp",1:"Annual ROI"})

In [ ]:
def get_coin_price(coin,vs_currency,days):
    data = requests.get("https://api.coingecko.com/api/v3/coins/" + 
                        coin +
                        "/market_chart?vs_currency=" +
                        vs_currency +
                        "&days=" + 
                        days)
    
    data_json = data.json() 
    price_list = list(np.array(data_json["prices"])[:,1])
    return price_list

In [ ]:
df = pd.DataFrame()
df["timestamp"] = (get_hist_apy
                   ("bd072651-d99c-4154-aeae-51f12109c054")
                   ["timestamp"])
df["ETH_price"] = get_coin_price("ethereum","USD",f"{len(df) - 1}")
df["Annual ROI, USDC-FRAX, %"] = (get_hist_apy
                                 ("bd072651-d99c-4154-aeae-51f12109c054")
                                 ["Annual ROI"])
df["Annual ROI, stETH-ETH, %"] = (get_hist_apy
                                 ("5ce23e7e-3800-4c9c-ad30-6db3db0515a1")
                                 ["Annual ROI"])
df["position_value"] = position_value(df,2)

In [ ]:
def strategy_PnL(df,HF):
    returns_list = []
    for i in range(len(df) - 1):
        USDC_init_price = 1 / df["ETH_price"][i]
        USDC_next_price = 1 / df["ETH_price"][i + 1]
        short_yearly_ROI = (
                            ((USDC_next_price / USDC_init_price) - 1) / 
                            (2 * HF)
                           )

        FRAXUSDC_yearly_ROI = df["Annual ROI, USDC-FRAX, %"][i] / (HF * 36500)
        stETHETH_yearly_ROI = df["Annual ROI, stETH-ETH, %"][i] / (HF * 36500)
        total_yearly_ROI = (
                           FRAXUSDC_yearly_ROI + 
                           stETHETH_yearly_ROI + 
                           short_yearly_ROI
                           )
        returns_list.append(total_yearly_ROI)
    
    return returns_list

In [ ]:
def position_value(df,HF):
    returns_list = strategy_PnL(df,2)
    cum_ret_list = []
    init_value = 1

    for i in range(len(returns_list)):
        init_value = init_value * (1 + returns_list[i])
        cum_ret_list.append(init_value)
    
    cum_ret_list.insert(0,1)
    
    return cum_ret_list

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x = df["timestamp"], 
               y = df["position_value"],
               name = "Position Value"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x = df["timestamp"], 
               y = df["ETH_price"],
               name = "ETH price"),
    secondary_y=True,
)
fig.update_layout(
    title = "SSETH return vs ETH move",
    xaxis_title = "Time",
    autosize = False,
    width = 1000,
    height = 500
)
fig.update_yaxes(title_text="ETH price", secondary_y = True)
fig.update_yaxes(title_text="Returns", secondary_y = False)

In [ ]:
def calc_return_df(pool_id):
    df = get_hist_apy(pool_id)
    df_2 = pd.DataFrame()
    index = range(0,105)
    df_2["PnL"] = (
                   pd.read_csv("/content/Traders Net PnL_2021-08-31_2022-11-11.csv")
                   [-len(df):]
                   ["Net PnL"]
                  )
    
    df_2["AUM"] = (
                   pd.read_csv("/content/AUM & Glp Supply_2021-08-31_2022-11-11.csv")
                   [-len(df):]
                   ["aum"] 
                  )
    
    df_2["new_index"] = list(range(0,len(df)))
    df_2.set_index("new_index",inplace = True)

    df["return"] =   - df_2["PnL"] / df_2["AUM"]
    df["ETH_price"] = get_coin_price("ethereum","USD","171")
    df["Annual ROI"] /= 100 
   
    
    return df

In [ ]:
pool_weights = {"stables":0.50,
                "ETH": 0.3,
                "BTC": 0.2,
               }

In [ ]:
def GLP_strategy_PnL(df,HF,pool_weights):
    returns_list = []
    for i in range(len(df) - 1):
        USDC_init_price = 1 / df["ETH_price"][i]
        USDC_next_price = 1 / df["ETH_price"][i + 1]
        divisor = (
                   pool_weights["stables"] / 
                   (pool_weights["stables"] 
                    + pool_weights["ETH"])
                  ) * 1 / HF
        
        short_ROI = ((USDC_next_price / USDC_init_price) - 1) / divisor
                            

        GLP_fee_ROI = df["Annual ROI"][i] / (HF * 365)
        GLP_PnL_ROI = df["return"][i] / HF
        total_ROI = (
                     GLP_fee_ROI + 
                     GLP_PnL_ROI + 
                     short_ROI
                    )
        returns_list.append(total_ROI)
    
    return returns_list

In [ ]:
def pos_value(df,HF,pool_weights):
    data_frame = df
    returns_list = GLP_strategy_PnL(df,HF,pool_weights)
    cum_ret_list = []
    init_value = 1

    for i in range(len(returns_list)):
        init_value = (init_value * 
                      (1 + returns_list[i] / 
                       (1 + pool_weights["BTC"] / 
                        HF
                       )
                      )
                     )
        cum_ret_list.append(init_value)
    
    cum_ret_list.insert(0,1)
    data_frame["cum_returns"] = cum_ret_list
    
    return df

In [ ]:
pos_value(calc_return_df("6a3ddb91-0638-4454-97c1-86dc6d59f32c"),2,pool_weights)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x = pos_value(calc_return_df
                             ("6a3ddb91-0638-4454-97c1-86dc6d59f32c"),
                             2,
                             pool_weights)
                             ["timestamp"], 
               y = pos_value(calc_return_df
                ("6a3ddb91-0638-4454-97c1-86dc6d59f32c"),
                2,
                pool_weights)
               ["cum_returns"],
               name = "Position Value"),
              secondary_y=False,
             )

fig.add_trace(
    go.Scatter(x = pos_value(calc_return_df
                             ("6a3ddb91-0638-4454-97c1-86dc6d59f32c"),
                             2,pool_weights)["timestamp"], 
               y = pos_value(calc_return_df
                             ("6a3ddb91-0638-4454-97c1-86dc6d59f32c"),
                             2,pool_weights)["ETH_price"],
               name = "ETH price"),
    secondary_y=True,
)
fig.update_layout(
    title = "SSETH (via GLP) return vs ETH move",
    xaxis_title = "Time",
    autosize = False,
    width = 1000,
    height = 500
)
fig.update_yaxes(title_text="ETH price", secondary_y = True)
fig.update_yaxes(title_text="Returns", secondary_y = False)